### Setting up the environment

In [1]:
import pickle
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, LSTM, RNN, Bidirectional, Flatten, Activation, \
    RepeatVector, Permute, Multiply, Lambda, Concatenate, BatchNormalization
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import SGD, Adam
from sklearn.metrics import confusion_matrix
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import TimeDistributed, GRU
import tensorflow.keras.backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import RMSprop, Adagrad
from transformers import TFBertModel
from tensorflow.keras.layers import SpatialDropout1D
from transformers import TFBertForSequenceClassification, BertConfig
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import f1_score, recall_score, precision_score
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.callbacks import EarlyStopping, ModelCheckpoint



In [2]:
import os
import tensorflow 

import random

seed_value = 42
#seed_value = None

np.random.seed(seed_value)
random.seed(seed_value)
tensorflow.random.set_seed(seed_value)

environment_name = sys.executable.split('/')[-3]
print('Environment:', environment_name)
os.environ[environment_name] = str(seed_value)
os.environ['PYTHONHASHSEED'] = str(seed_value)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import tensorflow.compat.v1.keras.backend as K
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

Environment: biotmpygpu


In [3]:
!nvidia-smi

Sun Apr 28 17:47:26 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.154.05             Driver Version: 535.154.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti     Off | 00000000:1E:00.0 Off |                  N/A |
| 31%   34C    P8              12W / 250W |    465MiB / 11264MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [6]:
#select_gpus = [0]
select_gpus = [0,1,2,3]    

In [7]:
import os
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

if select_gpus:
    devices = []
    for gpu in select_gpus:
        devices.append('/gpu:' + str(gpu))    
    strategy = tensorflow.distribute.MirroredStrategy(devices=devices)
    os.environ["CUDA_VISIBLE_DEVICES"] = ''

else:
    # Get the GPU device name.
    device_name = tensorflow.test.gpu_device_name()
    # The device name should look like the following:
    if device_name == '/device:GPU:0':
        print('Using GPU: {}'.format(device_name))
    else:
        raise SystemError('GPU device not found')

    os.environ["CUDA_VISIBLE_DEVICES"] = device_name
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

Num GPUs Available:  8
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')


### Dataset Loading

In [8]:
df = pd.read_csv("/home/resperanca/Projeto/KIBA.csv")
df = df.drop_duplicates()
df.to_pickle('KIBA.pkl')

In [11]:
# Criar um DataFrame contendo 'ProteinID' e 'compound_iso_smiles'
#df_smiles = df[['ProteinID', 'compound_iso_smiles']]

# Criar um DataFrame contendo 'ProteinID' e 'target_sequence'
#df_sequence = df[['ProteinID', 'target_sequence']]

# Salvar os dois DataFrames em arquivos CSV separados, se necessário
#df_smiles.to_csv('smiles.csv', index=False)  # Salvar o DataFrame de SMILES em um arquivo CSV
#df_sequence.to_csv('sequence.csv', index=False)  # Salvar o DataFrame de sequência em um arquivo CSV

# Salvar os dois DataFrames como arquivos pickle
#df_smiles.to_pickle('smiles.pkl')  # Salvar o DataFrame de SMILES em um arquivo pickle
#df_sequence.to_pickle('sequence.pkl') 

In [7]:
# with open('final_dataframe.pkl','rb') as file:
#     final_df = pickle.load(file)

In [8]:
#final_df_no_repeats = pd.read_csv('rheaDB_wNegatives.csv', index_col=0)

/home/malves/miniconda3/envs/biotmpygpu/lib/python3.8/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


## Encoding inputs

In [9]:
CHAR_PROT_SET = { "A": 1, "C": 2, "B": 3, "E": 4, "D": 5, "G": 6,
				"F": 7, "I": 8, "H": 9, "K": 10, "M": 11, "L": 12,
				"O": 13, "N": 14, "Q": 15, "P": 16, "S": 17, "R": 18,
				"U": 19, "T": 20, "W": 21,
				"V": 22, "Y": 23, "X": 24,
				"Z": 25 }

CHAR_SMILE_SET = {"#": 29, "%": 30, ")": 31, "(": 1, "+": 32, "-": 33, "/": 34, 
                 ".": 2, "1": 35, "0": 3, "3": 36, "2": 4, "5": 37, "4": 5, 
                 "7": 38, "6": 6, "9": 39, "8": 7, "=": 40, "A": 41, "@": 8, "C": 42, "B": 9, "E": 43, 
                 "D": 10, "G": 44, "F": 11, "I": 45, "H": 12, "K": 46, "M": 47, "L": 13, 
                 "O": 48, "N": 14, "P": 15, "S": 49, "R": 16, "U": 50, "T": 17, "W": 51, 
                 "V": 18, "Y": 52, "[": 53, "Z": 19, "]": 54, "\\": 20, "a": 55, "c": 56, 
                 "b": 21, "e": 57, "d": 22, "g": 58, "f": 23, "i": 59, "h": 24, "m": 60, 
                 "l": 25, "o": 61, "n": 26, "s": 62, "r": 27, "u": 63, "t": 28, "y": 64, '*':65}

In [11]:
PROT_LEN = 1198
SMILE_LEN = 176


def label_smiles(line, MAX_SMI_LEN, CHAR_SMILE_SET):
	X = np.zeros(MAX_SMI_LEN)
	for i, ch in enumerate(line[:MAX_SMI_LEN]): #	x, smi_ch_ind, y
		X[i] = CHAR_SMILE_SET[ch]

	return X #.tolist()

def label_sequence(line, MAX_SEQ_LEN, CHAR_PROT_SET):
	X = np.zeros(MAX_SEQ_LEN)

	for i, ch in enumerate(line[:MAX_SEQ_LEN]):
		X[i] = CHAR_PROT_SET[ch]

	return X #.tolist()

In [12]:
encoded_data = {'x_prot':[], 'x_met':[], 'y_train':[]}

for i, (index, row) in enumerate(df.iterrows()):
     print('percentage: {0:.3%}'.format(i/df.shape[0]), end='\r')
     encoded_data['x_prot'].append(label_sequence(row['target_sequence'], PROT_LEN, CHAR_PROT_SET))
     encoded_data['x_met'].append(label_smiles(row['compound_iso_smiles'], SMILE_LEN, CHAR_SMILE_SET))
     encoded_data['y_train'].append(row['ProteinID'])

In [13]:
encoded_data

{'x_prot': [array([11., 17.,  1., ...,  0.,  0.,  0.]),
  array([11., 18., 16., ...,  4., 23., 12.]),
  array([11.,  4., 12., ..., 23., 12., 20.]),
  array([11., 17.,  6., ...,  0.,  0.,  0.]),
  array([11., 17.,  6., ...,  0.,  0.,  0.]),
  array([11., 17., 12., ...,  0.,  0.,  0.]),
  array([11., 17.,  6., ...,  0.,  0.,  0.]),
  array([11., 17., 21., ...,  0.,  0.,  0.]),
  array([11.,  4., 18., ...,  0.,  0.,  0.]),
  array([11., 15., 17., ...,  0.,  0.,  0.]),
  array([11.,  1.,  5., ...,  0.,  0.,  0.]),
  array([11.,  4., 12., ...,  0.,  0.,  0.]),
  array([11., 16.,  5., ...,  0.,  0.,  0.]),
  array([11.,  1.,  6., ..., 16., 12., 12.]),
  array([11.,  6., 20., ...,  0.,  0.,  0.]),
  array([11.,  6., 14., ...,  0.,  0.,  0.]),
  array([11.,  6., 14., ...,  0.,  0.,  0.]),
  array([11.,  6., 14., ...,  0.,  0.,  0.]),
  array([11., 15., 10., ...,  0.,  0.,  0.]),
  array([11.,  6., 20., ...,  0.,  0.,  0.]),
  array([11.,  5., 18., ...,  0.,  0.,  0.]),
  array([11.,  1.,  6., 

In [23]:
#with open('nr_encoded_data.pkl', 'rb') as file:
     #encoded_data = pickle.load(file)

In [14]:
smaller_dataset = {'x_prot': encoded_data['x_prot'][:220_000],
                     'x_met': encoded_data['x_met'][:220_000],
                     'y_train':encoded_data['y_train'][:220_000]}

In [20]:
with open('clean_dataset.pkl', 'wb') as file:
    encoded_data = pickle.dump(smaller_dataset, file=file)

with open('clean_dataset.pkl', 'rb') as file:
    encoded_data = pickle.load(file)

In [21]:
with open('clean_dataset.pkl', 'rb') as file:
    encoded_data = pickle.load(file)

### Spliting data

In [18]:
x_prot = encoded_data['x_prot']
x_met = encoded_data['x_met']
y = encoded_data['y_train']

In [19]:
x_prot_train = np.array(x_prot[:200_000])
x_met_train = np.array(x_met[:200_000])
y_train = np.array(y[:200_000])

In [22]:
x_prot_val = np.array(x_prot[200_000:210_000])
x_met_val = np.array(x_met[200_000:210_000])
y_val = np.array(y[200_000:210_000])

In [23]:
x_prot_test = np.array(x_prot[210_000:])
x_met_test = np.array(x_met[210_000:])
y_test = np.array(y[210_000:])

### Creating the model

In [25]:
def DeepDTA(PROT_LEN, SMILE_LEN, CHAR_SMI_SET_SIZE, CHAR_PROT_SET_SIZE, NUM_FILTERS):
    X_PROT = Input(shape=(PROT_LEN,), dtype='int32')
    X_MET = Input(shape=(SMILE_LEN,), dtype='int32') ### Buralar flagdan gelmeliii


    ### SMI_EMB_DINMS  FLAGS GELMELII 
    encode_protein = Embedding(input_dim=CHAR_PROT_SET_SIZE, output_dim=128, input_length=PROT_LEN)(X_PROT)
    encode_protein = Conv1D(filters=NUM_FILTERS, kernel_size=4,  activation='relu', padding='valid',  strides=1)(encode_protein)
    encode_protein = Conv1D(filters=NUM_FILTERS*2, kernel_size=8,  activation='relu', padding='valid',  strides=1)(encode_protein)
    encode_protein = Conv1D(filters=NUM_FILTERS*3, kernel_size=12,  activation='relu', padding='valid',  strides=1)(encode_protein)
    encode_protein = GlobalMaxPooling1D()(encode_protein)
    
    encode_smiles = Embedding(input_dim=CHAR_SMI_SET_SIZE, output_dim=128, input_length=SMILE_LEN)(X_MET) 
    encode_smiles = Conv1D(filters=NUM_FILTERS, kernel_size=4,  activation='relu', padding='valid',  strides=1)(encode_smiles)
    encode_smiles = Conv1D(filters=NUM_FILTERS*2, kernel_size=6,  activation='relu', padding='valid',  strides=1)(encode_smiles)
    encode_smiles = Conv1D(filters=NUM_FILTERS*3, kernel_size=8,  activation='relu', padding='valid',  strides=1)(encode_smiles)
    encode_smiles = GlobalMaxPooling1D()(encode_smiles)


    encode_interaction = tf.keras.layers.concatenate([encode_smiles, encode_protein], axis=-1) #merge.Add()([encode_smiles, encode_protein])

    # Fully connected 
    FC1 = Dense(1024, activation='relu')(encode_interaction)
    FC2 = Dropout(0.1)(FC1)
    FC2 = Dense(1024, activation='relu')(FC2)
    FC2 = Dropout(0.1)(FC2)
    FC2 = Dense(512, activation='relu')(FC2)


    # And add a logistic regression on top
    predictions = Dense(1, activation='sigmoid')(FC2) #kernel_initializer='normal"

    interactionModel = Model(inputs=[X_PROT, X_MET], outputs=[predictions])

    interactionModel.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    print(interactionModel.summary())
    return interactionModel

In [26]:
if select_gpus:
    with strategy.scope():
        model = DeepDTA(PROT_LEN, SMILE_LEN, len(CHAR_SMILE_SET), len(CHAR_PROT_SET), 32)

else:
    model = DeepDTA(PROT_LEN, SMILE_LEN, len(CHAR_SMILE_SET), len(CHAR_PROT_SET), 32)


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 176)]        0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 1198)]       0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 176, 128)     8320        input_2[0][0]                    
__________________________________________________________________________________________________
embedding (Embedding)           (None, 1198, 128)    3200        input_1[0][0]                    
______________________________________________________________________________________________

In [27]:
keras_callbacks = [
        EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5),
       ModelCheckpoint('deepdta_checkpoint_clean_dataset.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
]

##### Model fit was performed using the run_deepdta_small.py file

In [28]:
history = model.fit([x_prot_train, x_met_train], 
 						y_train,
 						epochs=50, 
 						batch_size=256, 
 						validation_data=([x_prot_val, x_met_val], y_val),
 						callbacks=keras_callbacks)

Epoch 1/50
INFO:tensorflow:batch_all_reduce: 20 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3').
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3').
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0

In [20]:
with open('trainHistoryDictSmallData.pkl', 'rb') as file_pi:
        history = pickle.load(file_pi)

In [21]:
model.load_weights('deepdta_checkpoint_smallData.h5')

In [108]:
y_pred_test = model.predict([x_prot_test, x_met_test],verbose=0)

In [24]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score, matthews_corrcoef
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score, auc, roc_curve, precision_recall_curve
from sklearn.metrics import confusion_matrix

In [109]:
def print_metrics(y_test, y_pred_test):
    y_pred_test = y_pred_test[:, 0]

    yhat_classes = np.where(y_pred_test > 0.5, 1, y_pred_test)
    yhat_classes = np.where(yhat_classes < 0.5, 0, yhat_classes).astype(np.int64)
    # accuracy: (tp + tn) / (p + n)
    test_acc = accuracy_score(y_test, yhat_classes)
    print('Accuracy: %f' % test_acc)

    # precision tp / (tp + fp)
    test_prec = precision_score(y_test, yhat_classes)
    print('Precision: %f' % test_prec)

    # recall: tp / (tp + fn)
    test_recall = recall_score(y_test, yhat_classes)
    print('Recall: %f' % test_recall)

    # f1: 2 tp / (2 tp + fp + fn)
    test_f1_score = f1_score(y_test, yhat_classes)
    print('F1 score: %f' % test_f1_score)

    # # ROC AUC
    # print('ROC AUC: %f' % test_roc_auc)

    # # PR AUC
    # print('PR AUC: %f' % test_pr_auc)

    # kappa
    test_kappa = cohen_kappa_score(y_test, yhat_classes)
    print('Cohens kappa: %f' % test_kappa)

    test_mcc = matthews_corrcoef(y_test, yhat_classes)
    print('MCC: %f' % test_mcc)

    # confusion matrix
    matrix = confusion_matrix(y_test, yhat_classes)
    print('Confusion Matrix:\n %s \n' % matrix)
    
print_metrics(y_test, y_pred_test)


Accuracy: 0.976980
Precision: 0.975468
Recall: 0.978915
F1 score: 0.977188
Cohens kappa: 0.953956
MCC: 0.953962
Confusion Matrix:
 [[48392  1240]
 [ 1062 49306]] 



In [26]:
tn, fp, fn, tp = confusion_matrix(y_test, yhat_classes).ravel()
print(tn, fn, fp, tp)

48392 1062 1240 49306


In [46]:
embl_drugs = pd.read_excel('2019_Mapping_paper_drug_list.xlsx', sheet_name='DrugInfo')
embl_drugs

,MOLENAME,TherapeuticIndication,TargetedRT,TargetedMZ,DrugPoolNumbers,EstimatedColonConcentrationMaier2018uM,SMILES,STATUS,TradeName,cas,...,Number_of_thioether,Number_of_sulfonamides,Number_of_sulfone_groups,Number_of_terminal_acetylenes,Number_of_tetrazole_rings,Number_of_thiazole_rings,Number_of_thiocyanates,Number_of_thiophene_rings,Number_of_unbranched_alkanes_of_at_least_4_members,Number_of_urea_groups
0,ABACAVIR SULFATE,antiviral,1.67,286.154209,8 10 19 21,NaN,Nc1nc(NC2CC2)c2ncn(C3C=CC(CO)C3)c2n1,"USAN, INN, BAN",ZIAGEN,188062-50-2,...,0,0,0,0,0,0,0,0,0,0
1,ACEBUTOLOL,"antihypertensive, antianginal, antiarrhythmic",2.05,336.204908,2 4 17 20,NaN,CCCC(=O)Nc1ccc(OCC(O)C[NH2+]C(C)C)c(C(C)=O)c1,"USAN, INN, BAN",SECTRAL,"34381-68-5, 37517-30-9 [acebutolol]",...,0,0,0,0,0,0,0,0,0,0
2,ACECAINIDE,antiarrhythmic,1.34,277.179027,10 11 16 19,NaN,CC[NH+](CC)CCNC(=O)c1ccc(NC(C)=O)cc1,"USAN, INN",NAPA,"34118-92-8, 32795-44-1 [acecainide]",...,0,0,0,0,0,0,0,0,0,0
3,ALFUZOSIN,alpha(1)-adrenergic blocker,2.45,389.206305,3 7 8 13,54.000864,COc1cc2nc(N(C)CCCNC(=O)C3CCCO3)nc(N)c2cc1OC,"USAN, INN, BAN",UROXATRAL,81403-80-7,...,0,0,0,0,0,0,0,0,1,0
4,ALMOTRIPTAN,5HT 1B/2D receptor agonist,1.83,335.166749,9 10 15 16,NaN,C[NH+](C)CCc1c[nH]c2ccc(CS(=O)(=O)N3CCCC3)cc12,"USAN, INN, BAN",AXERT,"154323-57-6, 181183-52-8(malate)",...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,WARFARIN,"anticoagulant, rodenticide",4.24,308.104860,1 2 17 18,NaN,CC(=O)CC(c1ccccc1)c1c([O-])c2ccccc2oc1=O,"USP, INN, BAN, JAN",COUMADIN,"81-81-2, 2610-86-8 [warfarin potassium], 129-0...",...,0,0,0,0,0,0,0,0,0,0
267,ZALEPLON,"sedative, hypnotic",3.40,305.127660,8 13 15 21,NaN,CCN(C(C)=O)c1cccc(-c2ccnc3c(C#N)cnn23)c1,"USAN, INN, BAN",SONATA,151319-34-5,...,0,0,0,0,0,0,0,0,0,0
268,ZIDOVUDINE [AZT],"RT transferase inhibitor, antiviral",0.97,267.096755,3 7 10 20,NaN,Cc1cn(C2CC(N=[N+]=[N-])C(CO)O2)c(=O)[nH]c1=O,"USP, INN, BAN, JAN",RETROVIR,30516-87-1,...,0,0,0,0,0,0,0,0,0,0
269,ZIPRASIDONE MESYLATE,antipsychotic,2.79,412.112460,3 7 9 21,244.767706,O=C1Cc2cc(CC[NH+]3CCN(c4nsc5ccccc45)CC3)c(Cl)c...,USAN,GEODON,199191-69-0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
embl_genes = pd.read_excel('2019_Mapping_paper_confirmed_gene_list.xlsx', skiprows=3, sheet_name='Table S13')
embl_genes

,Gene,Unnamed: 1,Unnamed: 2,Unnamed: 3,Artemisinin,Artemisinin_208.1468,Artemisinin_236.1415,Bisacodyl,Bisacodyl_183.0685,Bisacodyl_277.1107,...,Phenazopyridine_160.0751,Phenazopyridine_174.0902,Racecadotril,Roxatidine acetate,Roxatidine acetate_306.1943,Sulfasalazine,Sulfasalazine_183.0799,Sulfasalazine_249.0576,Tinidazole,Tinidazole_345.082
0,RefSeq Locus Tag,PATRIC ID,Product,Protein ID,Parent drug,208.147,236.142,Parent drug,183.069,277.111,...,160.075,174.09,Parent drug,Parent drug,306.194,Parent drug,183.08,249.058,Parent drug,345.082
1,BT_0152,fig|226186.12.peg.149,acetyl esterase (acetylxylosidase),NP_809065.1,0,0.000,0.000,1,1.000,1.000,...,0.000,0.00,1,1,1.000,0,1.00,0.000,0,0.000
2,BT_0217,fig|226186.12.peg.216,NADH oxidase,NP_809130.1,0,0.000,0.000,0,0.000,0.000,...,0.000,0.00,0,0,0.000,1,0.00,1.000,1,0.000
3,BT_0445,fig|226186.12.peg.445,endoglucanase E precursor (EGE),NP_809358.1,0,0.000,0.000,1,0.000,1.000,...,0.000,0.00,0,0,0.000,0,0.00,0.000,0,0.000
4,BT_0569,fig|226186.12.peg.570,Lysophospholipase L1 and related esterases,NP_809482.1,0,0.000,0.000,1,1.000,1.000,...,0.000,0.00,1,1,1.000,0,1.00,0.000,0,0.000
5,BT_1006,fig|226186.12.peg.1022,Putative nitroreductase,NP_809919.1,0,0.000,0.000,0,0.000,0.000,...,0.000,0.00,0,0,0.000,0,0.00,0.000,1,0.000
6,BT_1148,fig|226186.12.peg.1170,Nitroreductase,NP_810061.1,0,0.000,0.000,0,0.000,0.000,...,0.000,0.00,0,0,0.000,0,0.00,0.000,1,0.000
7,BT_1192,fig|226186.12.peg.1215,putative xylanase,NP_810105.1,0,0.000,0.000,1,1.000,1.000,...,0.000,0.00,1,1,0.000,0,1.00,0.000,0,0.000
8,BT_1429,fig|226186.12.peg.1461,putative general stress protein,NP_810342.1,1,1.000,1.000,0,0.000,0.000,...,1.000,1.00,0,0,0.000,1,0.00,1.000,0,0.000
9,BT_2068,fig|226186.12.peg.2127,3-oxo-5-alpha-steroid 4-dehydrogenase,NP_810981.1,0,0.000,0.000,0,0.000,0.000,...,0.000,0.00,0,0,0.000,0,0.00,0.000,0,0.000


#### Getting protein sequences from NCBI

In [29]:
from Bio import Entrez, SeqIO

In [30]:
Entrez.email = 'pg36999@alunos.uminho.pt'

In [31]:

from Bio import Entrez

import time
Entrez.email ="eigtw59tyjrt403@gmail.com"


handle = Entrez.efetch(db="protein", id='NP_809065', rettype='fasta')


In [32]:
def get_prot_sequences(ids):
    sequences = []
    for ncbi_id in ids:
        with Entrez.efetch(
            db="protein", rettype="fasta", retmode="text", id=str(ncbi_id)) as handle:
            for seq_record in SeqIO.parse(handle, "fasta"):
                sequences.append(str(seq_record.seq))

    return sequences

In [33]:
ncbi_ids = embl_genes["Unnamed: 3"].iloc[1:,].values

In [34]:
protein_sequences = get_prot_sequences(ncbi_ids)

In [37]:
protein_sequences.insert(0,'Protein Sequences')

In [39]:
embl_genes['Protein Sequences'] = protein_sequences

In [40]:
embl_genes

,Gene,Unnamed: 1,Unnamed: 2,Unnamed: 3,Artemisinin,Artemisinin_208.1468,Artemisinin_236.1415,Bisacodyl,Bisacodyl_183.0685,Bisacodyl_277.1107,...,Phenazopyridine_174.0902,Racecadotril,Roxatidine acetate,Roxatidine acetate_306.1943,Sulfasalazine,Sulfasalazine_183.0799,Sulfasalazine_249.0576,Tinidazole,Tinidazole_345.082,Protein Sequences
0,RefSeq Locus Tag,PATRIC ID,Product,Protein ID,Parent drug,208.147,236.142,Parent drug,183.069,277.111,...,174.09,Parent drug,Parent drug,306.194,Parent drug,183.08,249.058,Parent drug,345.082,Protein Sequences
1,BT_0152,fig|226186.12.peg.149,acetyl esterase (acetylxylosidase),NP_809065.1,0,0.000,0.000,1,1.000,1.000,...,0.00,1,1,1.000,0,1.00,0.000,0,0.000,MKKKKLLLIALLLVGAASSFAAKVDTLLVKSPSMNKDIKVVVVTPD...
2,BT_0217,fig|226186.12.peg.216,NADH oxidase,NP_809130.1,0,0.000,0.000,0,0.000,0.000,...,0.00,0,0,0.000,1,0.00,1.000,1,0.000,MLSAAAFAADKVVKLPKPNLNRTGTVMKALSERQSTREYASKALTL...
3,BT_0445,fig|226186.12.peg.445,endoglucanase E precursor (EGE),NP_809358.1,0,0.000,0.000,1,0.000,1.000,...,0.00,0,0,0.000,0,0.00,0.000,0,0.000,MWKNVLVLVSLCLLFEDVNAQTTKNRYYKANDENVSYVGRTEIQSD...
4,BT_0569,fig|226186.12.peg.570,Lysophospholipase L1 and related esterases,NP_809482.1,0,0.000,0.000,1,1.000,1.000,...,0.00,1,1,1.000,0,1.00,0.000,0,0.000,MYMKKKRLGQWMLLAIVCLSFSVGETYAQKHEFANYKRYATENAAL...
5,BT_1006,fig|226186.12.peg.1022,Putative nitroreductase,NP_809919.1,0,0.000,0.000,0,0.000,0.000,...,0.00,0,0,0.000,0,0.00,0.000,1,0.000,MNLDEVLNYRRSVRVYDKEKEIDIEKVKHCLELATLAPNSSDMQLW...
6,BT_1148,fig|226186.12.peg.1170,Nitroreductase,NP_810061.1,0,0.000,0.000,0,0.000,0.000,...,0.00,0,0,0.000,0,0.00,0.000,1,0.000,MKTNEVLENIKARRSVRAYTDRQVSEEDLQAILEAATFAPSGMHLE...
7,BT_1192,fig|226186.12.peg.1215,putative xylanase,NP_810105.1,0,0.000,0.000,1,1.000,1.000,...,0.00,1,1,0.000,0,1.00,0.000,0,0.000,MKQKFSSLFVLLVLLLTGLQVSAQSTPKPFDIEQPSLRVFLPAPEL...
8,BT_1429,fig|226186.12.peg.1461,putative general stress protein,NP_810342.1,1,1.000,1.000,0,0.000,0.000,...,1.00,0,0,0.000,1,0.00,1.000,0,0.000,MSTKTMKEKAVELLQKCEVVTLASVNKEGYPRPVPMSKIAAEGIST...
9,BT_2068,fig|226186.12.peg.2127,3-oxo-5-alpha-steroid 4-dehydrogenase,NP_810981.1,0,0.000,0.000,0,0.000,0.000,...,0.00,0,0,0.000,0,0.00,0.000,0,0.000,MSIAAFNLFLGVMSLTALIVFIALYFVKAGYGIFRTASWGVAISNK...


In [41]:
# embl_genes.to_csv('genes_list_wProtSequences.csv')

In [42]:
embl_genes = pd.read_csv('genes_list_wProtSequences.csv', index_col=0)
embl_genes

,Gene,Unnamed: 1,Unnamed: 2,Unnamed: 3,Artemisinin,Artemisinin_208.1468,Artemisinin_236.1415,Bisacodyl,Bisacodyl_183.0685,Bisacodyl_277.1107,...,Phenazopyridine_174.0902,Racecadotril,Roxatidine acetate,Roxatidine acetate_306.1943,Sulfasalazine,Sulfasalazine_183.0799,Sulfasalazine_249.0576,Tinidazole,Tinidazole_345.082,Protein Sequences
0,RefSeq Locus Tag,PATRIC ID,Product,Protein ID,Parent drug,208.147,236.142,Parent drug,183.069,277.111,...,174.09,Parent drug,Parent drug,306.194,Parent drug,183.08,249.058,Parent drug,345.082,Protein Sequences
1,BT_0152,fig|226186.12.peg.149,acetyl esterase (acetylxylosidase),NP_809065.1,0,0.000,0.000,1,1.000,1.000,...,0.00,1,1,1.000,0,1.00,0.000,0,0.000,MKKKKLLLIALLLVGAASSFAAKVDTLLVKSPSMNKDIKVVVVTPD...
2,BT_0217,fig|226186.12.peg.216,NADH oxidase,NP_809130.1,0,0.000,0.000,0,0.000,0.000,...,0.00,0,0,0.000,1,0.00,1.000,1,0.000,MLSAAAFAADKVVKLPKPNLNRTGTVMKALSERQSTREYASKALTL...
3,BT_0445,fig|226186.12.peg.445,endoglucanase E precursor (EGE),NP_809358.1,0,0.000,0.000,1,0.000,1.000,...,0.00,0,0,0.000,0,0.00,0.000,0,0.000,MWKNVLVLVSLCLLFEDVNAQTTKNRYYKANDENVSYVGRTEIQSD...
4,BT_0569,fig|226186.12.peg.570,Lysophospholipase L1 and related esterases,NP_809482.1,0,0.000,0.000,1,1.000,1.000,...,0.00,1,1,1.000,0,1.00,0.000,0,0.000,MYMKKKRLGQWMLLAIVCLSFSVGETYAQKHEFANYKRYATENAAL...
5,BT_1006,fig|226186.12.peg.1022,Putative nitroreductase,NP_809919.1,0,0.000,0.000,0,0.000,0.000,...,0.00,0,0,0.000,0,0.00,0.000,1,0.000,MNLDEVLNYRRSVRVYDKEKEIDIEKVKHCLELATLAPNSSDMQLW...
6,BT_1148,fig|226186.12.peg.1170,Nitroreductase,NP_810061.1,0,0.000,0.000,0,0.000,0.000,...,0.00,0,0,0.000,0,0.00,0.000,1,0.000,MKTNEVLENIKARRSVRAYTDRQVSEEDLQAILEAATFAPSGMHLE...
7,BT_1192,fig|226186.12.peg.1215,putative xylanase,NP_810105.1,0,0.000,0.000,1,1.000,1.000,...,0.00,1,1,0.000,0,1.00,0.000,0,0.000,MKQKFSSLFVLLVLLLTGLQVSAQSTPKPFDIEQPSLRVFLPAPEL...
8,BT_1429,fig|226186.12.peg.1461,putative general stress protein,NP_810342.1,1,1.000,1.000,0,0.000,0.000,...,1.00,0,0,0.000,1,0.00,1.000,0,0.000,MSTKTMKEKAVELLQKCEVVTLASVNKEGYPRPVPMSKIAAEGIST...
9,BT_2068,fig|226186.12.peg.2127,3-oxo-5-alpha-steroid 4-dehydrogenase,NP_810981.1,0,0.000,0.000,0,0.000,0.000,...,0.00,0,0,0.000,0,0.00,0.000,0,0.000,MSIAAFNLFLGVMSLTALIVFIALYFVKAGYGIFRTASWGVAISNK...


In [47]:
embl_drugs

,MOLENAME,TherapeuticIndication,TargetedRT,TargetedMZ,DrugPoolNumbers,EstimatedColonConcentrationMaier2018uM,SMILES,STATUS,TradeName,cas,...,Number_of_thioether,Number_of_sulfonamides,Number_of_sulfone_groups,Number_of_terminal_acetylenes,Number_of_tetrazole_rings,Number_of_thiazole_rings,Number_of_thiocyanates,Number_of_thiophene_rings,Number_of_unbranched_alkanes_of_at_least_4_members,Number_of_urea_groups
0,ABACAVIR SULFATE,antiviral,1.67,286.154209,8 10 19 21,NaN,Nc1nc(NC2CC2)c2ncn(C3C=CC(CO)C3)c2n1,"USAN, INN, BAN",ZIAGEN,188062-50-2,...,0,0,0,0,0,0,0,0,0,0
1,ACEBUTOLOL,"antihypertensive, antianginal, antiarrhythmic",2.05,336.204908,2 4 17 20,NaN,CCCC(=O)Nc1ccc(OCC(O)C[NH2+]C(C)C)c(C(C)=O)c1,"USAN, INN, BAN",SECTRAL,"34381-68-5, 37517-30-9 [acebutolol]",...,0,0,0,0,0,0,0,0,0,0
2,ACECAINIDE,antiarrhythmic,1.34,277.179027,10 11 16 19,NaN,CC[NH+](CC)CCNC(=O)c1ccc(NC(C)=O)cc1,"USAN, INN",NAPA,"34118-92-8, 32795-44-1 [acecainide]",...,0,0,0,0,0,0,0,0,0,0
3,ALFUZOSIN,alpha(1)-adrenergic blocker,2.45,389.206305,3 7 8 13,54.000864,COc1cc2nc(N(C)CCCNC(=O)C3CCCO3)nc(N)c2cc1OC,"USAN, INN, BAN",UROXATRAL,81403-80-7,...,0,0,0,0,0,0,0,0,1,0
4,ALMOTRIPTAN,5HT 1B/2D receptor agonist,1.83,335.166749,9 10 15 16,NaN,C[NH+](C)CCc1c[nH]c2ccc(CS(=O)(=O)N3CCCC3)cc12,"USAN, INN, BAN",AXERT,"154323-57-6, 181183-52-8(malate)",...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,WARFARIN,"anticoagulant, rodenticide",4.24,308.104860,1 2 17 18,NaN,CC(=O)CC(c1ccccc1)c1c([O-])c2ccccc2oc1=O,"USP, INN, BAN, JAN",COUMADIN,"81-81-2, 2610-86-8 [warfarin potassium], 129-0...",...,0,0,0,0,0,0,0,0,0,0
267,ZALEPLON,"sedative, hypnotic",3.40,305.127660,8 13 15 21,NaN,CCN(C(C)=O)c1cccc(-c2ccnc3c(C#N)cnn23)c1,"USAN, INN, BAN",SONATA,151319-34-5,...,0,0,0,0,0,0,0,0,0,0
268,ZIDOVUDINE [AZT],"RT transferase inhibitor, antiviral",0.97,267.096755,3 7 10 20,NaN,Cc1cn(C2CC(N=[N+]=[N-])C(CO)O2)c(=O)[nH]c1=O,"USP, INN, BAN, JAN",RETROVIR,30516-87-1,...,0,0,0,0,0,0,0,0,0,0
269,ZIPRASIDONE MESYLATE,antipsychotic,2.79,412.112460,3 7 9 21,244.767706,O=C1Cc2cc(CC[NH+]3CCN(c4nsc5ccccc45)CC3)c(Cl)c...,USAN,GEODON,199191-69-0,...,0,0,0,0,0,0,0,0,0,0


#### Predicting values for Bisacodyl

In [144]:
smile_artimisinin = embl_drugs.loc[embl_drugs['MOLENAME'] == 'Bisacodyl'.upper()]['SMILES'].values[0]


In [145]:
y_real = embl_genes['Bisacodyl'].iloc[1:,].values.astype(int)

In [59]:
protein_sequences.pop(0)


'Protein Sequences'

In [146]:
encoded_proteins = []

for sequence in protein_sequences:
    encoded_proteins.append(label_sequence(sequence, 1198, CHAR_PROT_SET))

encoded_proteins = np.array(encoded_proteins)


In [147]:
encoded_smile = [label_smiles(smile_artimisinin, 176, CHAR_SMILE_SET)]


In [150]:
encoded_smile = np.array(encoded_smile*len(encoded_proteins))

In [153]:
y_real_pred = model.predict([encoded_proteins,encoded_smile], verbose=0)

In [154]:
y_real

array([1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0,
       0, 1, 0, 1, 1, 0, 1, 0])

In [155]:
print_metrics(y_real, y_real_pred)

Accuracy: 0.566667
Precision: 0.571429
Recall: 0.285714
F1 score: 0.380952
Cohens kappa: 0.101382
MCC: 0.115847
Confusion Matrix:
 [[13  3]
 [10  4]] 

